In [6]:
# delete all the hidden files
import os

# Define the directory where you want to delete hidden files
directory = '/Volumes/easystore/ERA5_single_level_2mTemp/'

# Loop through all files in the directory
for file_name in os.listdir(directory):
    # Check if the file starts with a dot (.), indicating it's a hidden file
    if file_name.startswith('.'):
        file_path = os.path.join(directory, file_name)
        try:
            # Check if it's a file (not a directory) and delete it
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted hidden file: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

print("All hidden files deleted.")


Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2022.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2021.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2023.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1982.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1966.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1978.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1980.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1965.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1985.nc
Deleted hidden file: /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_1986.nc
Deleted hidden file: /Volumes/easystore/ERA5_singl

In [14]:
# convert hourly to daily, and then regrid from 0.25 to 0.5
# if a year has been processed, skip that year

import os
import subprocess

# Define the input and output directories
input_dir = '/Volumes/easystore/ERA5_single_level_2mTemp/'  # Directory with your original .nc files
output_dir = '/Volumes/easystore/ERA5_single_level_2mTemp/output/'  # Directory where processed files will be saved

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the desired resolution and grid for regridding
target_grid = 'r360x180'  # 0.5 degree resolution

# Function to execute CDO commands
def process_file(input_file, output_file):
    try:
        # Create temporary file for daily mean data
        tmp_daily_file = os.path.join(output_dir, 'tmp_daily.nc')
        
        print(f"Processing {input_file} to daily!")
        # Step 1: Calculate daily means
        daymean_cmd = ['cdo', 'daymean', input_file, tmp_daily_file]
        subprocess.run(daymean_cmd, check=True)
        
        print(f"Processing {input_file} to 05deg!")
        # Step 2: Regrid the data from 0.25 degree to 0.5 degree
        # use remap_bil for non-conservative variables for a smoother transition
        # use remap_con when conserving the total sum (e.g., mass, fluxes) is important
        remap_cmd = ['cdo', 'remapbil,' + target_grid, tmp_daily_file, output_file]
        subprocess.run(remap_cmd, check=True)
        
        # Clean up the temporary file
        os.remove(tmp_daily_file)
        
        print(f"Processed {input_file} successfully!")
    
    except subprocess.CalledProcessError as e:
        print(f"Error processing {input_file}: {e}")

# Loop through all NetCDF files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.nc'):  # Only process .nc files
        input_file = os.path.join(input_dir, file_name)
        output_file = os.path.join(output_dir, f"processed_{file_name}")
        try:
            with open(output_file, 'r'):
                print(f"File {output_file} exists, skipping...")
                continue  # Move to the next file if it exists
        except FileNotFoundError:
            print(f"File {output_file} does not exist, processing...")
        
        process_file(input_file, output_file)


File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2020.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2021.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2022.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2019.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2024.nc does not exist, processing...
Processing /Volumes/easystore/ERA5_single_level_2mTemp/era5_2m_temperature_2024.nc to daily!


cdi  warning (cdfInqContents): Coordinates variable number can't be assigned!
cdi  warning (cdfInqContents): Coordinates variable expver can't be assigned!


Processing /Volumes/easystore/ERA5_single_level_2mTemp/era5_2m_temperature_2024.nc to 05deg!
cdo    remapbil: Bilinear weights from lonlat (1440x721) to lonlat (360x180) grid
Processed /Volumes/easystore/ERA5_single_level_2mTemp/era5_2m_temperature_2024.nc successfully!
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_2023.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_._era5_2m_temperature_2024.nc does not exist, processing...
Processing /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2024.nc to daily!


cdo    daymean: Open failed on >/Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2024.nc<
                Unsupported file type


Error processing /Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2024.nc: Command '['cdo', 'daymean', '/Volumes/easystore/ERA5_single_level_2mTemp/._era5_2m_temperature_2024.nc', '/Volumes/easystore/ERA5_single_level_2mTemp/output/tmp_daily.nc']' returned non-zero exit status 1.
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1982.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1966.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1978.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1968.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1975.nc exists, skipping...
File /Volumes/easystore/ERA5_single_level_2mTemp/output/processed_era5_2m_temperature_1977.nc exists, skipping...
File /Volumes/easy

cdo    daymean: Open failed on >/Volumes/easystore/ERA5_single_level_2mTemp/era5_2m_temperature_1991.nc<
                Unknown Error
